What if we only used the complete eukaryotic model to align all tRNAs? Then comparing identity elements across isotypes by position becomes much easier.

Also: track acceptor arm length, loop length, etc.

In [16]:
import pandas as pd
import subprocess
from tRNA_position import get_positions, position_generator

isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']
identities = pd.DataFrame()
for isotype in isotypes:
  # create new alignment file
  model = '/projects/lowelab/users/blin/tRNAscan/models/current/TRNAinf-euk.cm'
  fasta = '/projects/lowelab/users/blin/tRNAscan/models/1.6/fasta/euk-' + isotype + '-r2-031616.fa'
  alignment = 'alignments/euk-' + isotype + '.sto'
  subprocess.call('cmalign -g --notrunc --matchonly -o {} {} {} > /dev/null'.format(alignment, model, fasta), shell=True)
  
  # get positions
  positions = get_positions(alignment)
  
  # get identities
  df = pd.concat(pd.DataFrame({'Position': [position.position], 
                               'Symbol': [symbol],
                               'Frequency': [freq],
                               'Isotype': isotype,
                               'Clade': 'Mammalia'}) for position, symbol, freq in position_generator(positions))
  # combine into larger df
  identities = pd.concat([identities, df])
  
identities.head()

,Clade,Frequency,Isotype,Position,Symbol
0,Mammalia,0.946895,Ala,1:89,C:O
0,Mammalia,0.961296,Ala,2:88,C:O
0,Mammalia,0.959496,Ala,3:87,C:O
0,Mammalia,0.988749,Ala,4:86,C:O
0,Mammalia,0.957696,Ala,5:85,C:O


In [104]:
key_ides = identities.ix[identities.Frequency >= 0.98].groupby(["Position", "Symbol"])
key_ides = key_ides.apply(lambda x: ', '.join(x.Isotype)).to_frame()
#key_ides.axes
#key_ides.columns = key_ides.columns.get_level_values(0)


In [105]:
key_ides

0
Position Symbol                                                   
10:25    C:O                          Asp, Gly, Ile, Phe, Ser, Thr
         G:C                                                   Gln
         G:N     Ala, Arg, Asn, Glu, His, iMet, Leu, Lys, Met, ...
11:24    B:V                                                   Gly
         C:G                                                   Ser
         C:O                          Asp, Glu, Ile, Met, Trp, Val
         G:N                                         Cys, Lys, Phe
         Y:R                Ala, Arg, Gln, His, Leu, Pro, Thr, Tyr
12:23    B:V                               Ala, Cys, Ile, Pro, Val
         C:O                                         Asp, Met, Ser
         D:H                                                   Lys
         G:N                                              Gln, Leu
         H:D                                                   Thr
         S:S                                                   Glu
         U:A                                                   Phe
         W:C                                              Arg, His
         Y:R                                                   Tyr
13:22    G:N                                             iMet, Met
14       A       Ala, Arg, Asn, Asp, Cys, Gln, Glu, Gly, His, I...
15       G                      iMet, Leu, Met, Pro, Ser, Tyr, Val
         R       Ala, Arg, Asn, Asp, Cys, Gln, Glu, Gly, His, I...
16       B                                                Glu, Gly
         D                                                     His
         H                                     Arg, Ile, iMet, Ser
         Y                  Asn, Gln, Leu, Lys, Met, Phe, Tyr, Val
18       G       Ala, Arg, Asn, Asp, Cys, Gln, Glu, Gly, His, I...
         R                                                Lys, Pro
19       G       Ala, Arg, Asn, Asp, Cys, Gln, Glu, Gly, His, I...
1:89     B:V                                                   Gln
         C:O                          Arg, Asn, Ile, Trp, Tyr, Val
...                                                            ...
73       C                       Ala, Arg, Cys, Gln, His, Pro, Trp
         Y       Asn, Asp, Glu, Gly, Ile, iMet, Leu, Lys, Met, ...
74       R       Ala, Arg, Asn, Asp, Cys, Gln, Glu, Gly, His, I...
75       A       Ala, Arg, Asn, Asp, Cys, Gln, Gly, His, Ile, i...
         W                                                Glu, Lys
76       A                                                    iMet
         D                                                Trp, Val
         H                                 Ala, Asp, Gly, Phe, Tyr
         R                                                Leu, Ser
         W                                                     His
77       A                                                    iMet
         H                                           Ile, Met, Val
         U                                                Gly, His
         Y       Arg, Asn, Asp, Cys, Gln, Glu, Leu, Lys, Phe, P...
7:83     C:O     Ala, Asn, Asp, Cys, Glu, His, iMet, Met, Phe, ...
         D:H                                                   Gln
         W:C                                              Ile, Leu
8        U       Ala, Arg, Asn, Cys, Gln, Glu, Gly, His, Ile, i...
         Y                                                     Asp
9        D                                                     Phe
         G                                                Leu, Ser
         R       Ala, Arg, Asn, Asp, Cys, Gln, Gly, Ile, iMet, ...
         V                                                     His
90       A                                          iMet, Leu, Trp
         D                                      Arg, Gln, Lys, Thr
         H                                           Ala, Cys, Pro
         M                                                     Gly
         R                  Asn, Asp, Glu, Il

This is a great list, but I'd like to categorize isotypes into ranked groups, much like for bases and base pairs. 



Symbol  |  Isotypes  | Rule   | Rank


In [ ]:
key_ides[key_ides.Position == ""]

According to this list, we have several new identity elements. Here, I look at the ones that 

I'm cross-referencing Giegé et al. (1998), but there may be further work after this publication that identifies these IDEs.

Position  |  Isotype  | 